In [88]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.pipeline import make_pipeline,FunctionTransformer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import StandardScaler

In [2]:
laptop_data=pd.read_csv("laptop_price.csv",encoding="latin-1")

In [101]:
laptop_data["Product"]

0                               MacBook Pro
1                               Macbook Air
2                                    250 G6
3                               MacBook Pro
4                               MacBook Pro
                       ...                 
1298                         Yoga 500-14ISK
1299                         Yoga 900-13ISK
1300                     IdeaPad 100S-14IBR
1301    15-AC110nv (i7-6500U/6GB/1TB/Radeon
1302    X553SA-XX031T (N3050/4GB/500GB/W10)
Name: Product, Length: 1303, dtype: object

array(['MacBook Pro', 'Macbook Air', '250 G6', 'Aspire 3',
       'ZenBook UX430UN', 'Swift 3', 'Inspiron 3567', 'MacBook 12"',
       'IdeaPad 320-15IKB', 'XPS 13', 'Vivobook E200HA',
       'Legion Y520-15IKBN', '255 G6', 'Inspiron 5379',
       '15-BS101nv (i7-8550U/8GB/256GB/FHD/W10)', 'MacBook Air',
       'Inspiron 5570', 'Latitude 5590', 'ProBook 470', 'LapBook 15.6"',
       'E402WA-GA010T (E2-6110/2GB/32GB/W10)',
       '17-ak001nv (A6-9220/4GB/500GB/Radeon', 'IdeaPad 120S-14IAP',
       'Inspiron 5770', 'ProBook 450',
       'X540UA-DM186 (i3-6006U/4GB/1TB/FHD/Linux)', 'Inspiron 7577',
       'X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce', 'Aspire A515-51G',
       'Inspiron 7773', 'IdeaPad 320-15ISK', 'Rog Strix',
       'X751NV-TY001T (N4200/4GB/1TB/GeForce', 'Yoga Book', 'ProBook 430',
       'Inspiron 3576', '15-bs002nv (i3-6006U/4GB/128GB/FHD/W10)',
       'VivoBook Max', 'GS73VR 7RG',
       'X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux)', 'Vostro 5471',
       'IdeaPad 520S-1

In [100]:
transformed.toarray()

array([[1, 1, 1, ..., 1, 1, 1]])

In [3]:
laptop_data.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


In [4]:
laptop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 132.5+ KB


In [5]:
laptop_data.columns

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

In [109]:
#testing
def transformer(dataset):
    encoder=OrdinalEncoder(dtype=int)
    wt=[]
    ram=[]
    x_res=[]
    y_res=[]
    all_res=[]
    scrn_type=[]
    cpu_freq=[]
    cpu_type=[]
    for i in dataset.columns:
        X=dataset[i]
        if 'kg' in X[0]:
            for i in X:
                wt.append(float(i.split("kg")[0]))
            dataset.drop("Weight",axis=1,inplace=True)
            dataset["Weight(kg)"]=wt
        
        elif 'GB' in X[0]:
            for i in X:
                ram.append(int(i.split("GB")[0]))
            
            dataset.drop("Ram",axis=1,inplace=True)
            dataset["Ram(GB)"]=ram
    
        elif re.search(r"(\d{3,4}x\d{3,4})",X[0]):
            most_freq=dataset["ScreenResolution"].value_counts().keys()[0]
            for i in X:
                temp=(re.search(r"(\d{3,4}x\d{3,4})",i).group())
                temp2=i.split(temp)
                temp=temp.split('x')
                if temp2[0]=='':
                    scrn_type.append(most_freq)
                    x_res.append(int(temp[0]))
                    y_res.append(int(temp[1]))
                    all_res.append([None,int(temp[0]),int(temp[1])])
                else:
                    scrn_type.append(temp2[0])
                    x_res.append(int(temp[0]))
                    y_res.append(int(temp[1]))
                    all_res.append([temp2[0],int(temp[0]),int(temp[1])])
            dataset["X_Res"]=x_res
            dataset["Y_Res"]=y_res
            dataset["Screen_type"]=scrn_type
            dataset.drop("ScreenResolution",axis=1,inplace=True)
        elif 'GHz' in X[0]:
            for i in X:
                temp=re.search(r'(\d{1}.\d{0,2}GHz)',i)
                if temp:
                    temp0=re.split(r'(\d{1}.\d{0,2}GHz)',i)[0]
                    temp=temp.group().split('GHz')[0]
                    cpu_freq.append(temp)
                    cpu_type.append(temp[0])
                else:
                    temp0=re.split(r'(\d{1}GHz)',i)[1]
                    temp=re.search(r'(\d{1}GHz)',i).group().split('GHz')[0]
                    cpu_freq.append(temp)
                    cpu_type.append(temp0)
            dataset.drop("Cpu",axis=1,inplace=True)
            dataset["Cpu_freq(GHz)"]=cpu_freq
            dataset["Cpu_type(GHz)"]=cpu_type
        elif i not in ["Ram",'Cpu',"Weight","ScreenResolution"]:
            dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique())))      
            
    return dataset

In [63]:
# #testing
# def wt_transformer(X):
#     wt=[]
    
#     if 'kg' in X[0]:
#             for i in X:
#                 wt.append(float(i.split("kg")[0]))
#             return wt
       
# def ram_transformer(X): 
#         ram=[] 
#         if 'GB' in X[0]:
#             for i in X:
#                 ram.append(int(i.split("GB")[0]))
#         return ram
# def scr_transformer(X): 
#     scrn_type=[]
#     x_res=[]
#     y_res=[]   
#     if re.search(r"(\d{3,4}x\d{3,4})",X[0]):
#             most_freq=X.value_counts().keys()[0]
#             for i in X:
#                 temp=(re.search(r"(\d{3,4}x\d{3,4})",i).group())
#                 temp2=i.split(temp)
#                 temp=temp.split('x')
#                 if temp2[0]=='':
#                     scrn_type.append(most_freq)
#                     x_res.append(int(temp[0]))
#                     y_res.append(int(temp[1]))
#                 else:
#                     scrn_type.append(temp2[0])
#                     x_res.append(int(temp[0]))
#                     y_res.append(int(temp[1]))
#             return x_res,y_res,scrn_type
# def cpu_tansfomer(X):
#         cpu_freq=[]
#         cpu_type=[]
#         if 'GHz' in X[0]:
#             for i in X:
#                 temp=re.search(r'(\d{1}.\d{0,2}GHz)',i)
#                 if temp:
#                     temp0=re.split(r'(\d{1}.\d{0,2}GHz)',i)[0]
#                     temp=temp.group().split('GHz')[0]
#                     cpu_freq.append(temp)
#                     cpu_type.append(temp[0])
#                 else:
#                     temp0=re.split(r'(\d{1}GHz)',i)[1]
#                     temp=re.search(r'(\d{1}GHz)',i).group().split('GHz')[0]
#                     cpu_freq.append(temp)
#                     cpu_type.append(temp0)
#             return cpu_freq,cpu_type

In [111]:
to_check=laptop_data.copy()

In [112]:
to_check=transformer(to_check)

/tmp/ipykernel_646569/3191087560.py:66: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique())))


In [113]:
to_check

,laptop_ID,Company,Product,TypeName,Inches,Memory,Gpu,OpSys,Price_euros,X_Res,Y_Res,Screen_type,Cpu_freq(GHz),Cpu_type(GHz),Ram(GB),Weight(kg)
0,1,Apple,MacBook Pro,Ultrabook,13.3,128GB SSD,0,macOS,1339.69,2560,1600,IPS Panel Retina Display,2.3,2,8,1.37
1,2,Apple,Macbook Air,Ultrabook,13.3,128GB Flash Storage,1,macOS,898.94,1440,900,Full HD 1920x1080,1.8,1,8,1.34
2,3,HP,250 G6,Notebook,15.6,256GB SSD,2,No OS,575.00,1920,1080,Full HD,2.5,2,8,1.86
3,4,Apple,MacBook Pro,Ultrabook,15.4,512GB SSD,3,macOS,2537.45,2880,1800,IPS Panel Retina Display,2.7,2,16,1.83
4,5,Apple,MacBook Pro,Ultrabook,13.3,256GB SSD,4,macOS,1803.60,2560,1600,IPS Panel Retina Display,3.1,3,8,1.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,128GB SSD,9,Windows 10,638.00,1920,1080,IPS Panel Full HD / Touchscreen,2.5,2,4,1.80
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,512GB SSD,9,Windows 10,1499.00,3200,1800,IPS Panel Quad HD+ / Touchscreen,2.5,2,16,1.30
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,64GB Flash Storage,20,Windows 10,229.00,1366,768,Full HD 1920x1080,1.6,1,2,1.50
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1TB HDD,101,Windows 10,764.00,1366,768,Full HD 1920x1080,2.5,2,6,2.19


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'with_mean'

In [64]:
category_pipeline=make_pipeline(OneHotEncoder(),StandardScaler(with_mean=False))


In [67]:
ram_pipe=make_pipeline(FunctionTransformer(ram_transformer),StandardScaler())
cpu_pipe=make_pipeline(FunctionTransformer(cpu_tansfomer),StandardScaler())
scr_pipe=make_pipeline(FunctionTransformer(scr_transformer),StandardScaler())
wt_pipe=make_pipeline(FunctionTransformer(wt_transformer),StandardScaler())


In [68]:
laptop_data.columns

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

In [69]:
preprocessing=ColumnTransformer([("ram",ram_pipe,["Ram"]),("cpu",cpu_pipe,["Cpu"]),("scr",scr_pipe,["ScreenResolution"]),("weight",wt_pipe,["Weight"]),("cat",category_pipeline,make_column_selector(dtype_include=object)),("",StandardScaler(),make_column_selector(dtype_exclude=object)),],remainder=StandardScaler())



In [70]:
check=preprocessing.fit_transform(laptop_data)

KeyError: 0

In [104]:
to_check["Product"].replace(to_replace=to_check[""].unique(),value=range(len(to_check[""].unique())))

/tmp/ipykernel_646569/3770478808.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  to_check["Product"].replace(to_replace=to_check["Product"].unique(),value=range(len(to_check["Product"].unique())))


0         0
1         1
2         2
3         0
4         0
       ... 
1298    615
1299    404
1300    342
1301    616
1302    617
Name: Product, Length: 1303, dtype: int64